In [2]:
import pymongo
mongoUri = 'mongodb://{id}:{pw}@{endpoint}/{db}?authSource={db}'
client = pymongo.MongoClient(
    mongoUri.format(
        id="2020sw",
        pw="swswsw",
        endpoint="doctor.iptime.org",
        db="2020sw"
    ))
db = client.get_database()

print("Available collections:")
for c in db.list_collection_names():
    print("\t", c, db.get_collection(c).estimated_document_count())

Available collections:
	 articles 760535


In [3]:
col = db.get_collection("articles")
#col.update_many({"status": { "$exists":False }}, {"$set": {"status": 0}})
col.create_index([("status", pymongo.DESCENDING)])
col.create_index([("keywords", pymongo.DESCENDING)])

'keywords_-1'

In [4]:
from pprint import pprint
from datetime import datetime
import numpy as np

In [6]:
keyword_dict=dict()
count=0
dataSet=col.find({'date':datetime(2015,11,28,0,0)})
for i,doc in enumerate(dataSet):
    for j in doc['keywords']:
        if(j['name'] not in keyword_dict):
            keyword_dict[j['name']]=count
            count+=1
#print(keyword_dict)
key_list=[]
for i in keyword_dict:
    key_list.append(i)

In [5]:
def execRound(key_ndarray, lastRank, edgeSum):
    size=len(key_ndarray[0])
    newWeight=lastRank
    DF=0.85
    for i in range(size):
        pr=0.0
        
        for j in range(size):
            if (key_ndarray[i][j]==0): continue
            else :
                pr+=(lastRank[j]/edgeSum[j])

        newWeight[i]=((1-DF)/size)+(DF*pr)
    return newWeight

In [6]:
nodeNum=len(key_list)
key_ndarray=np.zeros((nodeNum,nodeNum))
pageRank=np.full(nodeNum,1/nodeNum)

for i,doc in enumerate(col.find({'date':datetime(2015,11,28,0,0)})):
    for j in doc['keywords']:
        init_name=j['name']
        for k in doc['keywords']:
            comp_name=k['name']
            if(init_name==comp_name):
                continue
            key_ndarray[keyword_dict[init_name]][keyword_dict[comp_name]]=1
            key_ndarray[keyword_dict[comp_name]][keyword_dict[init_name]]=1

edgeSum=key_ndarray[0]

for i in range(nodeNum):
    edgeSum[i]=key_ndarray[i].sum()
# pageRank=execRound(key_ndarray,pageRank)
pageRank=execRound(key_ndarray,pageRank,edgeSum)
print(pageRank.sum())

1.1982541807115052


In [11]:
pageRank=execRound(key_ndarray,pageRank,edgeSum)
print(pageRank.sum())

1.1704860839946798


In [12]:
pageRank=execRound(key_ndarray,pageRank,edgeSum)
print(pageRank.sum())

1.1358421672525012


In [13]:
print(pageRank)

[5.59865604e-03 1.22159127e-04 2.28012915e-04 ... 6.98178709e-05
 6.97715604e-05 6.97264649e-05]


In [16]:
Rank_list=[]
for i in range(nodeNum):
    Rank_list.append({'keyword':key_list[i],'pageRank':pageRank[i]})
Rank_list

[{'keyword': '영화', 'pageRank': 0.005598656044845374},
 {'keyword': '내부자', 'pageRank': 0.00012215912652025472},
 {'keyword': '청소년', 'pageRank': 0.0002280129149609499},
 {'keyword': '청소년관람불가', 'pageRank': 0.00012217478787743646},
 {'keyword': '관람', 'pageRank': 0.0002457306012614604},
 {'keyword': '불가', 'pageRank': 0.00012218906026816777},
 {'keyword': '청불', 'pageRank': 0.00012219857970924853},
 {'keyword': '등급', 'pageRank': 0.00024012788349485337},
 {'keyword': '등급영화', 'pageRank': 0.00012217254218935763},
 {'keyword': '가운데', 'pageRank': 0.0014989698476557964},
 {'keyword': '역대', 'pageRank': 0.00012216615551858778},
 {'keyword': '최단', 'pageRank': 0.0001221746879091602},
 {'keyword': '기간', 'pageRank': 0.0008220588574806233},
 {'keyword': '관객', 'pageRank': 0.0003438476411238166},
 {'keyword': '300', 'pageRank': 0.0005370723584930846},
 {'keyword': '300만명', 'pageRank': 0.0001221554890776803},
 {'keyword': '돌파', 'pageRank': 0.00012216160971505747},
 {'keyword': '배급사', 'pageRank': 0.0001221677

In [18]:
sorted_list=sorted(Rank_list, key=lambda keyword:(keyword['pageRank']),reverse=True)
print(sorted_list)

[{'keyword': '영화', 'pageRank': 0.005598656044845374}, {'keyword': '기자', 'pageRank': 0.0025399081613897987}, {'keyword': '뉴스', 'pageRank': 0.0023126441172516142}, {'keyword': '연합', 'pageRank': 0.002179874248306299}, {'keyword': '앵커', 'pageRank': 0.0021064730672930264}, {'keyword': '시간', 'pageRank': 0.002026492524522352}, {'keyword': '28', 'pageRank': 0.002016725196496985}, {'keyword': '미국', 'pageRank': 0.0020066765185946483}, {'keyword': '28일', 'pageRank': 0.0019793035475872316}, {'keyword': '연합뉴스', 'pageRank': 0.0019009909828110471}, {'keyword': '오늘', 'pageRank': 0.0018384499405913713}, {'keyword': '10', 'pageRank': 0.001760222125647123}, {'keyword': '27', 'pageRank': 0.0017602078699346067}, {'keyword': '이번', 'pageRank': 0.0017391522358751097}, {'keyword': '이후', 'pageRank': 0.0016927943520812644}, {'keyword': '우리', 'pageRank': 0.0016170954220400618}, {'keyword': '최근', 'pageRank': 0.001585269451318047}, {'keyword': '27일', 'pageRank': 0.001518472556557715}, {'keyword': '12', 'pageRank': 